<a href="https://colab.research.google.com/github/czengnn/lana-del-rey-lyrics-generator/blob/main/LDR_by_word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import pandas as pd 
import numpy as np 
import re 
import os
import time

import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.losses import sparse_categorical_crossentropy

from keras.models import Sequential
from keras.layers import LSTM, Activation, Flatten, Dropout, Dense, Embedding, TimeDistributed, Bidirectional
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [30]:
# load songs
songs = pd.read_csv('/content/drive/MyDrive/data/lana_lyrics_83.csv').drop('Unnamed: 0', axis=1)

# put lyrics into 1 string
text = ''
for song in songs['lyrics']:
  text = text + song.lower()

# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

Length of text: 143660 characters
65 unique characters


In [31]:
text.split('\n')[:10]

['[intro: lana del rey + sample]',
 'why? ("got that?")',
 'who, me? ("louder!")',
 'why? ("got that?")',
 '',
 '[verse 1]',
 "feet don't fail me now",
 'take me to the finish line',
 'oh, my heart, it breaks every step that i take',
 "but i'm hoping at the gates, they'll tell me that you're mine"]

In [32]:
corpus = list(set(text.split('\n')))

In [33]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

In [34]:
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [35]:
input_sequences[:5]

[[246, 301],
 [246, 301, 70],
 [246, 301, 70, 875],
 [246, 301, 70, 875, 1],
 [246, 301, 70, 875, 1, 206]]

In [36]:
# pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences,
                                         maxlen = max_sequence_len, padding='pre'))

In [37]:
input_sequences[:5]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0, 246, 301],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0, 246, 301,  70],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0, 246, 301,  70, 875],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
        246, 301,  70, 875,   1],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 246,
        301,  70, 875,   1, 206]], dtype=int32)

In [38]:
predictors, label = input_sequences[:,:-1], input_sequences[:,-1]

### Building The Model

In [67]:
def sparse_cat_loss(y_true,y_pred):
  return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

vocab_size = total_words
embed_dim = 50
def create_model(vocab_size, embed_dim):
    model = Sequential()
    model.add(Embedding(vocab_size, embed_dim, input_length=max_sequence_len-1))
    # Add an LSTM Layer
    model.add(Bidirectional(LSTM(150, return_sequences=True)))
    # A dropout layer for regularisation
    model.add(Dropout(0.2))
    # Add another LSTM Layer
    model.add(LSTM(100,return_sequences=False)) 
    model.add(Dense(vocab_size//2, activation='relu'))
    # In the last layer, the shape should be equal to the total number of words present in our corpus
    model.add(Dense(vocab_size, activation='softmax'))
    model.compile(loss=sparse_cat_loss, optimizer='adam', metrics='accuracy')  #(# Pick a loss function and an optimizer)
    return model
print(model.summary())


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 17, 50)            121000    
_________________________________________________________________
bidirectional_4 (Bidirection (None, 17, 300)           241200    
_________________________________________________________________
dropout_4 (Dropout)          (None, 17, 300)           0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense_8 (Dense)              (None, 1210)              122210    
_________________________________________________________________
dense_9 (Dense)              (None, 2420)              2930620   
Total params: 3,575,430
Trainable params: 3,575,430
Non-trainable params: 0
____________________________________________

In [47]:
history = model.fit(predictors, label, epochs=100, verbose=1)

Epoch 1/100
497/497 [==============================] - 62s 113ms/step - loss: 6.5682 - accuracy: 0.0360
Epoch 2/100
497/497 [==============================] - 56s 112ms/step - loss: 5.9237 - accuracy: 0.0370
Epoch 3/100
497/497 [==============================] - 58s 117ms/step - loss: 5.6952 - accuracy: 0.0394
Epoch 4/100
497/497 [==============================] - 56s 112ms/step - loss: 5.5221 - accuracy: 0.0532
Epoch 5/100
497/497 [==============================] - 56s 112ms/step - loss: 5.3240 - accuracy: 0.0684
Epoch 6/100
497/497 [==============================] - 56s 112ms/step - loss: 5.1168 - accuracy: 0.0889
Epoch 7/100
497/497 [==============================] - 56s 113ms/step - loss: 4.9135 - accuracy: 0.1033
Epoch 8/100
497/497 [==============================] - 56s 112ms/step - loss: 4.7251 - accuracy: 0.1210
Epoch 9/100
497/497 [==============================] - 56s 113ms/step - loss: 4.5724 - accuracy: 0.1317
Epoch 10/100
497/497 [==============================] - 56s 114m

In [68]:
# Save the weights
# model.save_weights('./ldr_lyrics_checkpoint')


# Create a new model instance
model_loaded = create_model(vocab_size, embed_dim)
model_loaded.load_weights('./ldr_lyrics_checkpoint')

In [70]:
def make_lyrics(model, seed_text, next_words):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list],
                     maxlen=max_sequence_len-1,padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [71]:
predicted_lyrics = make_lyrics(model_loaded, 'lolita', 100)
print(predicted_lyrics)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


lolita hey design single to me to me anyway down in the garden of evil beauty queen of the hollywood sign yeah i also call home when i'm not as rock them all work with to leave my head get crazy in the pale rocky what what what what what all newer can you are who get with you 'round here lately get with you on these beaches got a party we'll dance 'til dawn put you higher and higher and over there out of time past no might not aussi with you used rey just coke it so wow now


In [64]:
predicted_lyrics = make_lyrics('summer', 100)
print(predicted_lyrics)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


summer putting all the lights on and the beautiful people in it verse 1 lana del rey stevie nicks lana del rey stevie nicks carti a cult icys dancer but  del del del rey del del del del rey playboi carti a lights i i i del del del del del del del del del rey hop t t t t tender carti the summer remaining del del del del del del rey del del t nicks lana del rey stevie nicks lana del del del del del del del del del del del del del t del del del del
